In [1]:
import torch
import numpy as np
import scipy as sp
import math
import copy
from dataclasses import dataclass
from test_sampler import TestProblem

In [2]:
def nesterov(x_0, grad, L, mu, K):
    x_cur = x_0
    y_cur = x_0
    x_list = [x_0]
    y_list = [x_0]
    for i in range(K):
        x_upd = y_cur - (1/L)*(grad(y_cur, L))
        y_upd = x_upd + ((np.sqrt(L) - np.sqrt(mu))/(np.sqrt(L) + np.sqrt(mu))) * (x_upd - x_cur)


        x_list.append(x_upd)
        y_list.append(y_upd)

        x_cur = x_upd
        y_cur = y_upd

    return x_list, y_list

In [3]:
def grad_quad(x, L):
    A = np.array([[L,0],[0,1]])
    b = np.array([2,1])
    return A@x+b

In [4]:
# hyperparameters
# @dataclass
# class CFG:
#     x_0 = np.array([3, 1])
#     L = 100
#     K = 100000
#     mu = 1
# 
#     x_list_nes, y_list_nes = nesterov(x_0, grad_quad, L, mu, K)
# 
#     L_x = 10
#     L_y = 1000
#     mu_x = 0.1
#     mu_y = 0.1
#     A = np.array([[L_x, 0, 0, 0], [0, mu_x, 0, 0], [0, 0, L_y, 0], [0, 0, 0, mu_y]])
#     b = np.array([2, 1, 3, 1])
# 
#     # x_star = x_list_nes[-1]
#     x_star = np.linalg.solve(A, -b)
#     # x_0 = np.array([1, 2])
#     y_0 = np.array([2, 3])
#     n = 4
#     x_init = np.array(list(x_0) + list(y_0))
# 
#     d = 1 / 2 * (x_init.T @ A @ x_init + b.T @ x_init)
#     theta = np.linalg.norm(x_init - x_star)
#     alpha = 0.1
#     tau = 1 / (alpha * n * n + 1)
#     x_0 = np.array([3, 1])

In [5]:
test_problem = TestProblem()

In [11]:
# ACRCD


def ACRCD(x_0, y_0, K):
    x_list = [x_0]
    y_list = [y_0]

    x_cur = x_0
    y_cur = y_0

    z_cur_x = x_0
    z_cur_y = y_0

    q_cur_x = x_0
    q_cur_y = y_0
    
    L_x, L_y = 10, 10
    beta = 2


    n_ =  L_x ** beta + L_y ** beta
    
    # q_cur (code) = y (paper)
    for i in range(K):
        
        #####  redefine alpha, tau
        alpha = (i + 2) / (2 * n_ ** 2)
        tau = 2 / (i + 2)
        
        x_upd = tau*z_cur_x+(1-tau)*q_cur_x
        y_upd = tau*z_cur_y+(1-tau)*q_cur_y
        
        test_problem.x = torch.tensor(x_upd, requires_grad=True)
        test_problem.y = torch.tensor(y_upd, requires_grad=True)
        result, grad_x, grad_y = test_problem.calc()
        print(result)

        
        index_p = np.random.choice([0,1],p=[L_x ** beta / n_, 
                                            L_y ** beta / n_])
        
        if index_p == 0:
            q_upd_x = x_upd - (1/L_x)*grad_x
            q_upd_y = q_cur_y

        if index_p == 1:
            q_upd_y = y_upd - (1/L_y)*grad_y
            q_upd_x = q_cur_x

        if index_p == 0:
            z_upd_x = z_cur_x - (1/L_x)*alpha*n_*grad_x
            z_upd_y = z_cur_y

        if index_p == 1:
            z_upd_y = z_cur_y - (1/L_y)*alpha*n_*grad_y
            z_upd_x = z_cur_x



        x_list.append(x_upd)
        y_list.append(y_upd)

        x_cur = x_upd
        y_cur = y_upd

        z_cur_x = z_upd_x
        z_cur_y = z_upd_y

        q_cur_x = q_upd_x
        q_cur_y = q_upd_y


    return x_list, y_list


x_list_ACRCD, y_list_ACRCD = ACRCD(torch.rand(1000), torch.rand(2000, requires_grad=True), 100)

/var/folders/n9/tgs43kxj3nz79q8g8t2qzzzw0000gn/T/ipykernel_3164/1319893320.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_problem.x = torch.tensor(x_upd, requires_grad=True)
/var/folders/n9/tgs43kxj3nz79q8g8t2qzzzw0000gn/T/ipykernel_3164/1319893320.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_problem.y = torch.tensor(y_upd, requires_grad=True)


tensor(641.2473, grad_fn=<AddBackward0>)
tensor(-59550.5156, grad_fn=<AddBackward0>)
tensor(-752227.2500, grad_fn=<AddBackward0>)
tensor(-3758536., grad_fn=<AddBackward0>)
tensor(-27034492., grad_fn=<AddBackward0>)
tensor(-5.3367e+08, grad_fn=<AddBackward0>)
tensor(-1.4492e+10, grad_fn=<AddBackward0>)
tensor(-1.2709e+11, grad_fn=<AddBackward0>)
tensor(-1.8095e+12, grad_fn=<AddBackward0>)
tensor(-4.0944e+13, grad_fn=<AddBackward0>)
tensor(-1.3538e+15, grad_fn=<AddBackward0>)
tensor(-6.1043e+16, grad_fn=<AddBackward0>)
tensor(-7.5820e+17, grad_fn=<AddBackward0>)
tensor(-1.4083e+19, grad_fn=<AddBackward0>)
tensor(-3.4101e+20, grad_fn=<AddBackward0>)
tensor(-1.3362e+22, grad_fn=<AddBackward0>)
tensor(-1.6856e+23, grad_fn=<AddBackward0>)
tensor(-1.0297e+25, grad_fn=<AddBackward0>)
tensor(-2.6134e+26, grad_fn=<AddBackward0>)
tensor(-1.4309e+28, grad_fn=<AddBackward0>)
tensor(-4.3478e+29, grad_fn=<AddBackward0>)
tensor(-2.3880e+31, grad_fn=<AddBackward0>)
tensor(-1.0196e+33, grad_fn=<AddBackw